In [1]:
# Convert to slides using
#
#  jupyter nbconvert --to slides gppd-in-90-seconds.ipynb
#

import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd

# Handy trick for live coding, not for production
gppd.set_interactive()

# Quiet please
gp.setParam('OutputFlag', 0)

# Example data
import io
projects_csv = io.StringIO("project,resource\n0,1.1\n1,1.4\n2,1.2\n3,1.1\n4,0.9\n5,1.3\n6,1.0\n7,1.6\n8,1.7\n9,0.9\n10,1.5\n11,1.1\n12,1.2\n13,1.7\n14,0.4\n15,0.4\n16,0.3\n17,1.5\n18,1.5\n19,1.6\n20,1.8\n21,1.5\n22,1.0\n23,1.5\n24,0.5\n25,1.3\n26,0.5\n27,1.7\n28,1.1\n29,0.9\n")
projects = pd.read_csv(projects_csv).assign(project=lambda df: df['project'].apply("p{}".format)).set_index('project')
teams_csv = io.StringIO("team,capacity\n0,2.4\n1,1.8\n2,1.1\n3,1.9\n4,1.4\n")
teams = pd.read_csv(teams_csv).assign(team=lambda df: df['team'].apply("t{}".format)).set_index('team')
project_values_csv = io.StringIO("project,team,profit\n0,4,0.4\n1,4,1.3\n2,0,1.7\n2,1,1.7\n2,2,1.7\n2,3,1.7\n2,4,1.7\n3,4,1.3\n4,0,1.3\n4,1,1.3\n4,2,1.3\n4,3,1.3\n4,4,1.3\n5,0,1.8\n5,1,1.8\n5,2,1.8\n5,3,1.8\n5,4,1.8\n6,0,1.2\n6,1,1.2\n6,2,1.2\n6,3,1.2\n6,4,1.2\n7,3,0.9\n7,4,0.9\n8,3,1.0\n8,4,1.0\n9,4,1.2\n10,0,0.8\n10,1,0.8\n10,2,0.8\n10,3,0.8\n10,4,0.8\n11,0,1.3\n11,1,1.3\n11,2,1.3\n11,3,1.3\n11,4,1.3\n12,3,0.8\n12,4,0.8\n13,0,1.5\n13,1,1.5\n13,2,1.5\n13,3,1.5\n13,4,1.5\n14,3,1.7\n14,4,1.7\n15,4,1.3\n16,4,0.3\n17,0,1.2\n17,1,1.2\n17,2,1.2\n17,3,1.2\n17,4,1.2\n18,3,1.3\n18,4,1.3\n19,3,1.8\n19,4,1.8\n20,3,1.6\n20,4,1.6\n21,3,1.1\n21,4,1.1\n22,4,0.4\n23,4,1.0\n24,4,0.3\n25,0,1.0\n25,1,1.0\n25,2,1.0\n25,3,1.0\n25,4,1.0\n26,4,1.8\n27,3,0.8\n27,4,0.8\n28,0,1.0\n28,1,1.0\n28,2,1.0\n28,3,1.0\n28,4,1.0\n29,3,1.3\n29,4,1.3\n")
project_values = pd.read_csv(project_values_csv).assign(
    team=lambda df: df['team'].apply("t{}".format),
    project=lambda df: df['project'].apply("p{}".format),
).set_index(["project", "team"])

## gurobipy-pandas: a quick example

- We have a set of projects, each requiring resources
- We have a set of teams, each with some capacity
- Goal: allocate projects to maximize revenue

In [2]:
projects.head(3)  # w_i 

,resource
project,
p0,1.1
p1,1.4
p2,1.2


In [3]:
teams.head(3)  # c_j

,capacity
team,
t0,2.4
t1,1.8
t2,1.1


## Decision variables

- Define $x_{ij} \in \lbrace 0, 1 \rbrace$. If team $j$ completes project $i$, then $x_{ij} = 1$. If team $j$ completes project $i$, we profit $p_{ij}$. Maximize profit!

$$
\max \sum_{i \in I} \sum_{j \in J} p_{i} x_{ij}
$$

In [4]:
# gurobipy-pandas creates variables as a series
model = gp.Model()
model.ModelSense = GRB.MAXIMIZE
assignments = project_values.gppd.add_vars(
    model, vtype=GRB.BINARY, obj="profit", name="x"
)
assignments.head()  # p_ij & x_ij

profit                      x
project team                               
p0      t4       0.4  <gurobi.Var x[p0,t4]>
p1      t4       1.3  <gurobi.Var x[p1,t4]>
p2      t0       1.7  <gurobi.Var x[p2,t0]>
        t1       1.7  <gurobi.Var x[p2,t1]>
        t2       1.7  <gurobi.Var x[p2,t2]>

## Capacity constraint

- Assigned projects are limited by team capacity

$$
\sum_{i \in I} w_{i} x_{ij} \le c_{j} \quad \forall j \in J
$$

In [5]:
# gurobipy-pandas adds one constraint per grouped constraint
capacity_constraints = gppd.add_constrs(
    model,
    (projects["resource"] * assignments["x"]).groupby("team").sum(),
    GRB.LESS_EQUAL,
    teams["capacity"],
    name='capacity',
)
capacity_constraints.apply(model.getRow).head()

team
t0    1.2 x[p2,t0] + 0.9 x[p4,t0] + 1.3 x[p5,t0] + x...
t1    1.2 x[p2,t1] + 0.9 x[p4,t1] + 1.3 x[p5,t1] + x...
t2    1.2 x[p2,t2] + 0.9 x[p4,t2] + 1.3 x[p5,t2] + x...
t3    1.2 x[p2,t3] + 0.9 x[p4,t3] + 1.3 x[p5,t3] + x...
t4    1.1 x[p0,t4] + 1.4 x[p1,t4] + 1.2 x[p2,t4] + 1...
Name: capacity, dtype: object

## Allocate once

- Each project is allocated to at most one team

$$
\sum_{j \in J} x_{ij} \le 1 \quad \forall i \in I
$$

In [6]:
allocate_once = gppd.add_constrs(
    model, assignments['x'].groupby('project').sum(),
    GRB.LESS_EQUAL, 1.0, name="allocate_once",
)
allocate_once.apply(model.getRow).head()

project
p0                                              x[p0,t4]
p1                                              x[p1,t4]
p10    x[p10,t0] + x[p10,t1] + x[p10,t2] + x[p10,t3] ...
p11    x[p11,t0] + x[p11,t1] + x[p11,t2] + x[p11,t3] ...
p12                                x[p12,t3] + x[p12,t4]
Name: allocate_once, dtype: object

## Solutions

- Gurobi solves the model, allocating projects to maximize profit
- Solution values retrieved as a series on our original index

In [7]:
model.optimize()
(
    assignments["x"].gppd.X.to_frame()
    .query("x >= 0.9").reset_index()
    .groupby("team").agg({"project": list})
)

,project
team,
t0,"[p4, p5]"
t1,[p2]
t2,[p11]
t3,"[p6, p29]"
t4,"[p14, p15, p26]"
